# Vehicle Routing Problem (VRP) Genetic Algorithm

This notebook demonstrates a complete VRP solution using genetic algorithms with the DEAP library.
The project is organized into modular components for better maintainability and reusability.

## 1. Import Required Modules

In [38]:
# Import project modules
from locations import generate_random_locations, create_depot
from vrp_solver import VRPSolver
from tuning import VRPTuner, compare_tuning_methods
from plotting import plot_vrp_routes, plot_tuning_results, plot_parameter_correlation, create_vrp_summary_dashboard
from config import DEFAULT_VRP_CONFIG, DEFAULT_GA_CONFIG, TUNING_RANGES

# Import other required libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import random

print("All modules imported successfully!")

All modules imported successfully!


## 2. Setup VRP Problem

In [39]:
# Generate random locations
num_locations = 25
num_vehicles = 3

locations = generate_random_locations(
    num_locations=num_locations,
    x_range=DEFAULT_VRP_CONFIG["location_range_x"],
    y_range=DEFAULT_VRP_CONFIG["location_range_y"],
    seed=DEFAULT_VRP_CONFIG["seed"]
)

depot = create_depot(*DEFAULT_VRP_CONFIG["depot"])

print(f"Generated {num_locations} locations")
print(f"Depot location: {depot}")
print(f"Number of vehicles: {num_vehicles}")

Generated 25 locations
Depot location: (550, 470)
Number of vehicles: 3


## 3. Initialize VRP Solver

In [40]:
# Create VRP solver instance
solver = VRPSolver(locations, depot, num_vehicles)
print("VRP Solver initialized!")

VRP Solver initialized!


## 4. Solve VRP with Default Parameters

In [41]:
# Solve with default parameters
print("Solving VRP with default parameters...")
best_individual, population, stats, hof = solver.solve(
    pop_size=DEFAULT_GA_CONFIG["pop_size"],
    cxpb=DEFAULT_GA_CONFIG["cxpb"],
    mutpb=DEFAULT_GA_CONFIG["mutpb"],
    ngen=50,  
    tournsize=DEFAULT_GA_CONFIG["tournsize"],
    seed=DEFAULT_GA_CONFIG["seed"]
)

print(f"Best distance found: {best_individual.fitness.values[0]:.2f}")
print(f"Balance penalty: {best_individual.fitness.values[1]:.2f}")

Solving VRP with default parameters...
gen	nevals	avg    	min   	max  
0  	300   	7259.65	31.911	16879
1  	221   	6989.54	27.5136	15749
2  	234   	6849.91	54.2453	16348.2
3  	229   	6715.69	21.5077	15881.1
4  	222   	6683.43	31.4458	15878.7
5  	218   	6697.17	10.3276	16778.5
6  	236   	6694.47	66.0436	16391.4
7  	239   	6666.93	20.6252	15547.2
8  	242   	6652.23	13.9027	16698.1
9  	226   	6537.68	49.1859	15171.2
10 	230   	6479.69	23.4545	16021  
11 	237   	6568.93	70.6329	15852.6
12 	217   	6495.95	59.0362	15808.8
13 	234   	6433.55	52.506 	15401  
14 	222   	6371.99	19.7221	15666.6
15 	226   	6342.64	23.5997	15481.8
16 	218   	6251.28	70.1448	16108.1
17 	232   	6282.05	67.077 	15521.6
18 	232   	6266.66	69.9603	16120.9
19 	223   	6256.67	41.3293	16592  
20 	222   	6207.76	21.1349	15725.1
21 	224   	6080.11	25.5165	15681.3
22 	228   	6022.57	76.8815	15573  
23 	228   	6071.4 	55.0358	15443.5
24 	253   	6119.46	42.7294	15382.8
25 	213   	6013.3 	55.0405	15613.8
26 	211   	5879.03	45.73

## 5. Visualize Default Solution

In [42]:
# Plot the solution
fig = plot_vrp_routes(locations, depot, best_individual, num_vehicles, 
                     title="VRP Solution with Default Parameters")
fig.show()

## 6. Parameter Tuning

In [43]:
# Initialize tuner
tuner = VRPTuner(locations, depot, num_vehicles)

# Perform random search tuning
print("Starting parameter tuning...")
best_params, best_tuned_individual, best_distance, tuning_results = tuner.random_search_tuning(
    n_trials=10,  
    param_ranges=TUNING_RANGES,
    ngen=50
)

Starting parameter tuning...
Starting parameter tuning with 10 trials...
Trial 1/10: Distance = 9043.8
Trial 2/10: Distance = 7800.6
Trial 3/10: Distance = 6545.1
Trial 4/10: Distance = 9522.4
Trial 5/10: Distance = 8279.9
Trial 6/10: Distance = 9191.7
Trial 7/10: Distance = 7343.3
Trial 8/10: Distance = 5974.9
Trial 9/10: Distance = 7194.7
Trial 10/10: Distance = 6783.2

Best distance: 5974.9
Best parameters: {'pop_size': 276, 'cxpb': 0.8294806890002202, 'mutpb': 0.097805486090942, 'tournsize': 4}


## 7. Visualize Tuned Solution

In [49]:
# Plot the tuned solution
fig = plot_vrp_routes(locations, depot, best_tuned_individual, num_vehicles, 
                     title=f"VRP Solution with Tuned Parameters (Distance: {best_distance:.1f})")
fig.show()

## 8. Compare Results

In [50]:
# Compare default vs tuned results
default_distance = best_individual.fitness.values[0]
tuned_distance = best_distance
improvement = ((default_distance - tuned_distance) / default_distance) * 100

print("=== RESULTS COMPARISON ===")
print(f"Default GA Distance: {default_distance:.2f}")
print(f"Tuned GA Distance: {tuned_distance:.2f}")
print(f"Improvement: {improvement:.2f}%")
print(f"Best Parameters: {best_params}")

=== RESULTS COMPARISON ===
Default GA Distance: 6859.98
Tuned GA Distance: 5974.91
Improvement: 12.90%
Best Parameters: {'pop_size': 276, 'cxpb': 0.8294806890002202, 'mutpb': 0.097805486090942, 'tournsize': 4}


## 9. Tuning Results Visualization

In [51]:
# Plot tuning progress
fig = plot_tuning_results(tuning_results)
fig.show()

In [47]:
# Plot parameter correlation
fig = plot_parameter_correlation(tuning_results)
fig.show()

## 10. Advanced Analysis

In [52]:
# Analyze vehicle route statistics
routes = solver.get_vehicle_routes(best_tuned_individual)

route_stats = []
for i, route in enumerate(routes):
    distance = sum(np.linalg.norm(np.array(route[j+1]) - np.array(route[j])) 
                  for j in range(len(route)-1))
    num_stops = len(route) - 2  # Exclude depot start and end
    route_stats.append({
        'vehicle': i + 1,
        'distance': distance,
        'num_stops': num_stops,
        'avg_distance_per_stop': distance / max(num_stops, 1)
    })

stats_df = pd.DataFrame(route_stats)
print("Vehicle Route Statistics:")
print(stats_df)

Vehicle Route Statistics:
   vehicle     distance  num_stops  avg_distance_per_stop
0        1  1684.574443          9             187.174938
1        2  2142.813774          8             267.851722
2        3  2147.523226          8             268.440403
